In [ ]:
from GoogleNews import GoogleNews
import pandas as pd
from datetime import datetime

# Company symbols and name mapping
companies = {
    "MSFT": "Microsoft",
    "GOOG": "Google",
    "TSLA": "Tesla",
    "NVDA": "Nvidia",
    "AMZN": "Amazon"
}

# Fetch news for each company
for symbol, name in companies.items():
    print(f"Searching for news about {name}...")

    googlenews = GoogleNews(lang='en', period='custom')
    googlenews.set_time_range('01/01/2025', '04/30/2025')
    googlenews.search(f"{name} AI investment")

    results = googlenews.results()
    df = pd.DataFrame(results)

    if not df.empty:
        # Remove unnecessary columns, format date
        df = df[['title', 'date', 'link']]
        df.rename(columns={'date': 'Date', 'title': 'title', 'link': 'link'}, inplace=True)

        # Convert dates to datetime format for sorting
        def parse_date(date_str):
            try:
                return datetime.strptime(date_str, "%b %d, %Y")
            except:
                return pd.NaT

        df['Date'] = df['Date'].apply(parse_date)
        df.dropna(subset=['Date'], inplace=True)
        df.sort_values(by='Date', inplace=True)

        # Add company and month columns
        df.insert(0, 'company', symbol)
        df.insert(1, 'month', df['Date'].dt.strftime('%B'))

        # Reorder columns
        df = df[['company', 'month', 'Date', 'title', 'link']]

        # Save to Excel
        df.to_excel(f"{symbol}_news.xlsx", index=False)
        print(f"{symbol} news saved successfully.")
    else:
        print(f"No news found for {symbol}.")